In [2]:
%pip install -U mne-icalabel
%pip install onnxruntime  -U
%pip install asrpy
%pip install autoreject

  Using cached mne_icalabel-0.7.0-py3-none-any.whl.metadata (10 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached mne_icalabel-0.7.0-py3-none-any.whl (21.6 MB)
   ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.4 MB 2.1 MB/s eta 0:00:04
   ----- ---------------------------------- 1.0/7.4 MB 2.2 MB/s eta 0:00:03
   -------- ------------------------------- 1.6/7.4 MB 2.0 MB/s eta 0:00:03
   ----------- ---------------------------- 2.1/7.4 MB 2.1 MB/s eta 0:00:03
   -------------- ------------------------- 2.6/7.4 MB 2.2 MB/s eta 0:00:03
   ----------------- ---------------------- 3.1/7.4 MB 2.3 MB/s eta 0:00:02
   ------------------- -------------------- 3.7/7.4 MB 2.3 MB/s eta 0:00:02
   --------------------- ------------------ 3.9/7.4 MB 2.2 MB/s eta 0:00:02
   ------------------------ --------------- 4.5/7.4 MB 2.2 MB/s et

In [11]:
# --- 0. 套件與路徑設定 -------------------------------------------
import pathlib, mne, pandas as pd
ROOT   = pathlib.Path.cwd().parent          # D:\COGBCI
RAWDIR = ROOT / 'data' / 'raw'       # 你存放 .set/.fdt 的資料夾
SUBS   = [f'sub-{i:02d}' for i in range(1, 30)]
SESS   = ['ses-S1', 'ses-S2', 'ses-S3']
CONDS  = {'zeroBack':'0b', 'oneBack':'1b', 'twoBack':'2b'}

In [12]:
# --- 1. 快速列出第一層檔名，確認 .set/.fdt 是否齊全 ---------------
missing = []
for sub in SUBS:
    for ses in SESS:
        for c in CONDS.keys():
            stem = f'{c}.set'
            fset = RAWDIR / sub / ses / stem
            ffdt = fset.with_suffix('.fdt')
            if not (fset.exists() and ffdt.exists()):
                missing.append(str(fset))
                
print(f'❌ 缺檔數：{len(missing)}')
if missing:
    print('缺少清單：', *missing, sep='\n')

❌ 缺檔數：0


In [13]:
# --- 2. 範例讀一檔，跑最小處理 ------------------------------------
demo_path = RAWDIR / 'sub-01' / 'ses-S1' / 'zeroBack.set'
raw = mne.io.read_raw_eeglab(demo_path, preload=True)
raw.filter(1., 40.)
print(raw)                          # 確認 64 通道 / 500 Hz

Reading D:\COGBCI\data\raw\sub-01\ses-S1\zeroBACK.fdt
Reading 0 ... 198287  =      0.000 ...   396.574 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 s)



C:\Users\tp6mo\AppData\Local\Temp\ipykernel_11620\2588670109.py:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(demo_path, preload=True)
C:\Users\tp6mo\AppData\Local\Temp\ipykernel_11620\2588670109.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(demo_path, preload=True)
C:\Users\tp6mo\AppData\Local\Temp\ipykernel_11620\2588670109.py:3: RuntimeWarning: Not setting position of 1 ecg channel found in montage:
['ECG1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_eeglab(demo_path, preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


<RawEEGLAB | zeroBACK.fdt, 63 x 198288 (396.6 s), ~95.4 MiB, data loaded>


[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:    0.2s finished


In [15]:
# --- 0. 套件與路徑設定 -------------------------------------------
import pathlib, mne
from mne_icalabel import label_components

ROOT   = pathlib.Path('D:/COGBCI')
raw_f  = ROOT / 'data' / 'raw' / 'sub-01' / 'ses-S1' / 'zeroBACK.set'

# 1) 讀檔並直接刪除 ECG
raw = mne.io.read_raw_eeglab(raw_f, preload=True)
raw.drop_channels(['ECG1'])            # 1) 先移除 ECG
raw.set_eeg_reference('average', projection=True)  # 2) CAR

Reading D:\COGBCI\data\raw\sub-01\ses-S1\zeroBACK.fdt
Reading 0 ... 198287  =      0.000 ...   396.574 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


C:\Users\tp6mo\AppData\Local\Temp\ipykernel_11620\3455198105.py:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(raw_f, preload=True)
C:\Users\tp6mo\AppData\Local\Temp\ipykernel_11620\3455198105.py:9: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(raw_f, preload=True)
C:\Users\tp6mo\AppData\Local\Temp\ipykernel_11620\3455198105.py:9: RuntimeWarning: Not setting position of 1 ecg channel found in montage:
['ECG1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_eeglab(raw_f, preload=True)


<RawEEGLAB | zeroBACK.fdt, 62 x 198288 (396.6 s), ~93.9 MiB, data loaded>

In [ ]:
import pathlib, mne, pandas as pd, itertools, os
from mne_icalabel import label_components
from tqdm.notebook import tqdm  # 進度條（可選）

ROOT     = pathlib.Path(r'D:\COGBCI')
RAW_DIR  = ROOT / 'data' / 'raw'
PREP_DIR = ROOT / 'data' / 'preproc'
RES_DIR  = ROOT / 'results'
PREP_DIR.mkdir(exist_ok=True, parents=True)
RES_DIR.mkdir(exist_ok=True)

subs   = [f'sub-{i:02d}' for i in range(1, 30)]   # 01–29
sess   = ['ses-S1', 'ses-S2', 'ses-S3']
conds  = {'zeroBack':'0b', 'oneBack':'1b', 'twoBack':'2b'}  # 檔名→簡碼
all_cats = ['brain','muscle','eye','heart','line_noise','chan_noise','other']

summary = []  # 收集每檔的 ICLabel count

for sub, ses, cond in tqdm(itertools.product(subs, sess, conds), total=29*3*3):
    fset = RAW_DIR / sub / ses / f'{cond}.set'
    if not fset.exists():
        print(f"缺檔：{fset}"); continue
    
    # 1. 讀檔
    raw = mne.io.read_raw_eeglab(fset, preload=True, verbose='ERROR')
    if 'ECG1' in raw.ch_names:
        raw.drop_channels(['ECG1'])            # 2. 移除 ECG
    raw.set_eeg_reference('average', projection=True)  # 3. CAR
    raw.set_montage('standard_1020', match_case=False)
    raw.filter(1., 100., picks='eeg', verbose='ERROR') # 4. 1–100 Hz
    
    # 5. ICA
    ica = mne.preprocessing.ICA(
        n_components=None,
        method='infomax',
        fit_params=dict(extended=True),
        random_state=97,
        max_iter='auto',
        verbose='ERROR'
    ).fit(raw)
    
    # 6. ICLabel
    labels_dict = label_components(raw, ica, method='iclabel')
    pred = labels_dict['labels']              # 長度 = n_IC
    # 7. 排除非 brain IC
    ica.exclude = [idx for idx, lab in enumerate(pred) if lab != 'brain']
    raw_clean = ica.apply(raw.copy())
    
    # 統計類別數量
    cnts = pd.Series(pred).value_counts().reindex(all_cats, fill_value=0)
    summary.append({
        'sub': sub, 'ses': ses, 'cond': conds[cond], **cnts.to_dict()
    })
    
    # 8. 最終 1–40 Hz
    raw_clean.filter(1., 40., picks='eeg', verbose='ERROR')
    
    # 9. Epoch & baseline
    events, event_id = mne.events_from_annotations(raw_clean, verbose=False)
    epochs = mne.Epochs(
    raw_clean, events, event_id=event_id,
    tmin=-0.2, tmax=0.8, baseline=(-0.2, 0),
    picks='eeg', preload=True,
    event_repeated='drop',
    verbose=False
    )
    # 10. 儲存
    out_name = f'{sub}_{ses}_{conds[cond]}-epo.fif'
    epochs.save(PREP_DIR / out_name, overwrite=True, verbose=False)

# 將 ICLabel 統計寫出
df_summary = pd.DataFrame(summary)
df_summary.to_csv(RES_DIR / 'ICLabel_counts_all.csv', index=False)
print("✅ 批次完成，Epoch 檔與 ICLabel 統計已存入 data/preproc 與 results/")

  0%|          | 0/261 [00:00<?, ?it/s]

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 62 components
Computing Extended Infomax ICA


In [17]:
import mne
from mne_icalabel import label_components
import pandas as pd

ROOT   = pathlib.Path(r'D:\COGBCI')
RAWDIR = ROOT / 'data' / 'raw' / 'sub-01' / 'ses-S1'
FILENAME = 'zeroBACK.set'

# 1) 真正的 Raw：完全不濾波、不 ASR
raw_raw = mne.io.read_raw_eeglab(RAWDIR / FILENAME, preload=True)
raw_raw.drop_channels(['ECG1'])    # 如果要一開始就移除 ECG
raw_raw.set_eeg_reference('average', projection=True)
raw_raw.set_montage('standard_1020')

ica_raw = mne.preprocessing.ICA(
    n_components=30,
    method='infomax',
    fit_params=dict(extended=True),
    random_state=97
).fit(raw_raw)

labels_raw = label_components(raw_raw, ica_raw, method='iclabel')
df_raw = pd.DataFrame({'pred_label': labels_raw['labels']})
counts_raw = df_raw['pred_label'].value_counts().reindex(
    ['brain','muscle','eye','heart','line_noise','chan_noise','other'], 
    fill_value=0
).rename_axis('label').reset_index(name='count')
counts_raw.to_csv(ROOT / 'results' / 'sub-01_ICLabel_raw_true.csv', index=False)

FileNotFoundError: input_fname does not exist: "D:\COGBCI\notebooks\data\raw\sub-01\ses-S1\zeroBACK.set"

In [ ]:
# 2) Band-pass (1–100 Hz) 之後
raw_bp = mne.io.read_raw_eeglab(RAWDIR / FILENAME, preload=True)
raw_bp.drop_channels(['ECG1'])
raw_bp.set_eeg_reference('average', projection=True)
raw_bp.set_montage('standard_1020')
raw_bp.filter(1., 100., picks='eeg')

ica_bp = mne.preprocessing.ICA(
    n_components=30,
    method='infomax',
    fit_params=dict(extended=True),
    random_state=97
).fit(raw_bp)

labels_bp = label_components(raw_bp, ica_bp, method='iclabel')
df_bp = pd.DataFrame({'pred_label': labels_bp['labels']})
counts_bp = df_bp['pred_label'].value_counts().reindex(
    ['brain','muscle','eye','heart','line_noise','chan_noise','other'],
    fill_value=0
).rename_axis('label').reset_index(name='count')
counts_bp.to_csv(ROOT / 'notebooks' / 'results' / 'sub-01_ICLabel_bandpass.csv', index=False)

In [ ]:
import pathlib
import mne
import pandas as pd
from autoreject import AutoReject
from mne_icalabel import label_components

# ---------- 路徑與參數設定 ----------
ROOT     = pathlib.Path(r'D:\COGBCI')
RAW_DIR  = ROOT / 'notebooks' / 'data' / 'raw' / 'sub-01' / 'ses-S1'
FILENAME = 'zeroBack.set'
RES_DIR  = ROOT / 'notebooks' / 'results'
RES_DIR.mkdir(exist_ok=True)

# ---------- 1. 讀取並做 Band-pass (1–100 Hz) ----------
raw_bp_asr = mne.io.read_raw_eeglab(RAW_DIR / FILENAME, preload=True, verbose='ERROR')
if 'ECG1' in raw_bp_asr.ch_names:
    raw_bp_asr.drop_channels(['ECG1'])
raw_bp_asr.set_eeg_reference('average', projection=True)
raw_bp_asr.set_montage('standard_1020', match_case=False)
raw_bp_asr.filter(1., 100., picks='eeg', verbose='ERROR')

# ---------- 2. 用 autoreject 進行 artifact‐correction ----------
# 2-1. 先把 Band-pass 後的 raw 切成 Epochs
events, event_id = mne.events_from_annotations(raw_bp_asr, verbose=False)
epochs_ar = mne.Epochs(
    raw_bp_asr.copy(),
    events,
    event_id=event_id,
    tmin=-0.2,
    tmax=0.8,
    baseline=(-0.2, 0),
    picks='eeg',
    preload=True,
    verbose=False
)

# 2-2. 用 AutoReject 自動檢測並重插值壞 epoch
ar = AutoReject(n_interpolate=[1, 2], consensus=[0.1, 0.3])
epochs_clean = ar.fit_transform(epochs_ar)

# 2-3. 把重插值後的 epochs 重組回 Raw 結構
data_clean = epochs_clean.get_data().transpose(1, 2, 0).reshape(
    len(raw_bp_asr.ch_names), -1
)
raw_asr = mne.io.RawArray(data_clean, raw_bp_asr.info.copy())
raw_asr.set_montage(raw_bp_asr.get_montage(), match_case=False)
raw_asr.set_eeg_reference('average', projection=True)

# ---------- 3. ICA + ICLabel 與統計各類別數量 ----------
raw_asr.filter(1., 40., picks='eeg')
ica_bp_asr = mne.preprocessing.ICA(
    n_components=30,
    method='infomax',
    fit_params=dict(extended=True),
    random_state=97,
    max_iter='auto',
    verbose='ERROR'
).fit(raw_asr)

print("実際的 n_components_ =", ica_bp_asr.n_components_)
pred_bp_asr = labels_bp_asr['labels']
print("pred_bp_asr 長度 =", len(pred_bp_asr))
print("pred_bp_asr sample =", pred_bp_asr[:10])

labels_bp_asr = label_components(raw_asr, ica_bp_asr, method='iclabel')
pred_bp_asr   = labels_bp_asr['labels']

all_cats = ['brain','muscle','eye','heart','line_noise','chan_noise','other']
df_bp_asr = pd.DataFrame({'pred_label': pred_bp_asr})
all_cats = [
    'brain',
    'muscle artifact',
    'eye blink',
    'heart',
    'line noise',
    'channel noise',
    'other'
]
counts_bp_asr = (
    df_bp_asr['pred_label']
    .value_counts()
    .reindex(all_cats, fill_value=0)
    .rename_axis('label')
    .reset_index(name='count')
)

# ---------- 4. 存成 CSV ----------
out_fname = RES_DIR / 'sub-01_ICLabel_bandpass_asr.csv'
counts_bp_asr.to_csv(out_fname, index=False)
print(f"ICLabel (BP+ASR) 統計已存到：{out_fname}")
print(counts_bp_asr)
